In [4]:
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.mlab as mlab
%matplotlib inline
%precision??

In [0]:
# Импортируем датасет и избавимся от нулевых строк
df = pd.read_csv('framingham.csv')
df.dropna(axis=0,inplace=True) #избавляемся от строчек с пропущенными значениями

Задача 1:

прогноз риска ишемичсекой болезни сердца в 10 летней перспективе в зависимости от всех имеющихся параметров

In [0]:
# разбиваем датафрейм на две части, dfx - параметры, dfy - целевая переменная. 
dfx = df.drop('TenYearCHD', axis = 1)
dfy = df[['TenYearCHD']] 

In [0]:
# разбиваем датасет на train и test выборку в соотношениии 80% train / 20% test случайным образом
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.2) 

In [0]:
# используем логистическую регрессию из sklearn. Имя lm - для краткости записи
lm = linear_model.LogisticRegression(solver='liblinear') 
# solver='liblinear' задает алгоритм поиска максимума функции правдоподобия. 

In [0]:
# функция fit обучает регрессию - подбирает коэффициенты
model = lm.fit(X_train, y_train.values.ravel()) 

In [0]:
# выведем коэффициенты модели
model.coef_

In [0]:
# выведем коэффициент при константе
model.intercept_

In [0]:
# возьмем одного пациента из тестовой выборки и сделаем для него прогноз
X_test[:1]

In [0]:
# прогноз вероятности отнесения к каждому классу (0 - нет риска, 1 - есть риск)
# первый элемент массива - вероятность класса 0, второй - вероятность класса 1
lm.predict_proba(X_test[:1])

In [0]:
# классификация (0 - нет риска, 1 - есть риск)
lm.predict(X_test[:1])

In [0]:
# сделаем prediction классов на всей тестовой выборке
y_pred = lm.predict(X_test)
y_pred

In [0]:
y_pred.sum()

In [0]:
 # смотрим accuracy модели
lm.score(X_test, y_test)

In [0]:
# смотрим confusion matrix - таблицу правильных и неправильных предсказаний
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix 

In [0]:
y_test['TenYearCHD'].value_counts()

In [0]:
y_pred.sum()

In [0]:
# строим тепловую карту
# Обратите внимание - по строкам расположены настоящие значения классов, а по столбцам - предсказанные
sn.heatmap(cnf_matrix, annot=True)

Выводы:
Наша модель хорошо работает на "здоровых" пациентах и плохо на "больных":
Из 100 (примерно) пациентов группы риска модель нашла около 10 (примерно), это значит, что вероятность ошибки второго рода высока. Для медицинского теста это плохо, поэтому нам нужно более аккуратно выбирать параметры.

Задача 2

прогноз риска ишемической болезни сердца в 10 летней перспективе в зависимости только от пола

Делаем те же самые шаги, но с новыми переменными

In [0]:
list_col=['male']
dfx1 = df[list_col]
dfy1 = df[['TenYearCHD']]

X_train1, X_test1, y_train1, y_test1 = train_test_split(dfx1, dfy1, test_size=0.2)
lm1 = linear_model.LogisticRegression(solver='liblinear') 
model1 = lm1.fit(X_train1, y_train1.values.ravel())

In [0]:
# Выведем коэффициенты модели. 
model1.coef_

In [0]:
# выведем коэффициент при константе
model1.intercept_

In [0]:
# Сделаем предсказания класса для всей тестовой выборки и вычислим accuracy модели

y_pred1 = lm1.predict(X_test1) 
lm1.score(X_test1, y_test1)

In [0]:
# Посмотрим на ошибки предсказания более детально:
cnf_matrix2 = metrics.confusion_matrix(y_test1, y_pred1)
cnf_matrix2

In [0]:
sn.heatmap(cnf_matrix2, annot=True)

Выводы:
Acuuracy модели довольно высока, но если присмотреться, нас ждет сюрприз:
Внезапно модель считает всех пациентов здоровыми. Таким образом, пол сам по себе - бесполезный фактор для классификации.